# One-Beat Prediction

For each of the single-beat ECG summaries, use the neural network to
make a prediction. Compare this to the prediction obtained from
running full ECGs through the neural network.

In [ ]:
import sys
sys.path.append("../")

import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import resnet

In [ ]:
from constants import (
    DATA_DIR,
    MODEL_DIR,
    N_LEADS,
)

In [ ]:
# array with zero padding
array = np.load(f"{DATA_DIR}/one_beat_array.npy")
print(array.shape)

# Corresponding exam IDs
exam_ids = np.load(f"{DATA_DIR}/one_beat_exam_ids.npy")
print(exam_ids.shape)

# Read in exam metadata and limit to exam IDs where we have the one-beat summaries.
df = pd.read_csv(f'{DATA_DIR}/exams.csv')
df = df.loc[df['exam_id'].isin(exam_ids)]
print(df.shape)

In [ ]:
# Brute force sort df to match order of exam_ids
# For some reason, the other way of doing the sort (with a list
# comprehension) isn't working here.
df_sorted = []
for exam_id in exam_ids:
    df_sorted.append(df[df['exam_id'] == exam_id])
df = pd.concat(df_sorted)
df.head()

In [ ]:
# Visually check that this ordering matches the df above.
exam_ids[:5]

In [ ]:
# Instantiate the model using the config.json information.
config = f'{MODEL_DIR}/config.json'
with open(config, 'r') as f:
    config_dict = json.load(f)
model = resnet.ResNet1d(
    input_dim=(N_LEADS, config_dict['seq_length']),
    blocks_dim=list(zip(config_dict['net_filter_size'], config_dict['net_seq_lengh'])),
    n_classes=1,
    kernel_size=config_dict['kernel_size'],
    dropout_rate=config_dict['dropout_rate']
)


In [ ]:
# Retrieve the state dict, which has all the coefficients
state_dict = (torch.load('../model/model.pth',
              weights_only=False,
              map_location=torch.device('cpu')))

In [ ]:
# Load the state dict and set the model to eval mode.
model.load_state_dict(state_dict['model'])
model.eval()

In [ ]:
# Get the predictions
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
data_array = torch.Tensor(array)
data_array = data_array.transpose(-1, -2)
model.zero_grad()
y_pred = model(data_array)

In [ ]:
# Histogram of the predictions
plt.hist(y_pred.detach().numpy())
plt.show()

In [ ]:
# Scatterplot of the predictions against the predictions from
# the neural network on the full ECGs.
df['y_pred'] = y_pred.detach().numpy()
plt.scatter(df['nn_predicted_age'], df['y_pred'], alpha = 0.1)
plt.show()

In [ ]:
# Correlation of the predictions from the one-beat summaries
# and the predictions from the full ECGs.
np.corrcoef(df['nn_predicted_age'], df['y_pred'])